# Requirements

In [6]:
import pysurfline
import pandas as pd
import os
from twilio.rest import Client
from twilio_config import *
import time

# Set surf conditions

In [4]:
spotId = "640a4d90e92030fa8aa15c69"
wind = 'Offshore'
max_height = 2.5
min_height = 2
sunrise = 8
sunset = 18

# API connection

In [2]:
def wave_sms(spotId, wind, max_height, min_height, sunrise, sunset):

    spotforecasts = pysurfline.get_spot_forecasts(
        spotId,
        days=4,
        intervalHours=3,
    )

    df = spotforecasts.get_dataframe()

    # Comparar condiciones
    df_sesh = df[
        (df['surf_max'] <= max_height) & (df['surf_max'] >= min_height) &
        (df['timestamp_dt'].dt.hour <= sunset) & (df['timestamp_dt'].dt.hour >= sunrise) &
        (df['directionType'] != wind)
    ].copy()

    df_sesh['date'] = df_sesh['timestamp_dt'].dt.date
    df_sesh['hour'] = df_sesh['timestamp_dt'].dt.hour

    # Data para el sms
    df_sms = df_sesh[['hour', 'surf_max', 'directionType']]

    # SMS a string
    sms = df_sms.to_string(index=False)

    return sms


# Send Data with Twilio

In [7]:
time.sleep(2)
account_sid = TWILIO_ACCOUNT_SID 
auth_token = TWILIO_AUTH_TOKEN

client = Client(account_sid, auth_token)

message = client.messages \
                .create(
                     body='\nHola! \n\n\n Hoy puede salir ola en Buchu.' + wave_sms(spotId, wind, max_height, min_height, sunrise, sunset),
                     from_=PHONE_NUMBER,
                     to='+56982091549'
                 )

print('Mensaje Enviado ' + message.sid)

Mensaje Enviado SM351403a5c6a0d75b2006b62771dc4fc6
